In [11]:
# Import the dependencies
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras import layers
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Bidirectional, Embedding, Dropout
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json
import pickle
print("Imported dependencies.")

Imported dependencies.


In [32]:
#Define functions (also for importing)
def readdata(filename):
    return pd.read_csv(filename, names = ["message", "Intent"],encoding = "latin1")

def cleaning(sentences):
    words = []
    for s in sentences:
        clean = re.sub(r'[^ a-z A-Z 0-9]', " ", s)
        w = word_tokenize(clean)
        #stemming
        words.append([i.lower() for i in w])
    return words

def create_tokenizer(words, filters = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~'):
    token = Tokenizer(filters = filters)
    token.fit_on_texts(words)
    return token

def maximum_length(words):
    return(len(max(words, key = len)))

def encoding_doc(token, words):
    return(token.texts_to_sequences(words))

def padding_doc(encoded_doc, max_length):
    return(pad_sequences(encoded_doc, maxlen = int(max_length), padding = "post"))

def one_hot(encode):
    o = OneHotEncoder(sparse = False)
    return(o.fit_transform(encode))

def create_model(vocab_size, max_length):
    model = Sequential()
    model.add(Embedding(vocab_size, 128, input_length = max_length, trainable = False))
    model.add(Bidirectional(LSTM(128)))
    #model.add(LSTM(128))
    model.add(Dense(32, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(intentsize, activation = "softmax"))
    return model

def predictions(word_tokenizer, text, model, max_length=30):
    clean = re.sub(r'[^ a-z A-Z 0-9]', " ", text)
    test_word = word_tokenize(clean)
    test_word = [w.lower() for w in test_word]
    test_ls = word_tokenizer.texts_to_sequences(test_word)
    #Check for unknown words
    if [] in test_ls:
        test_ls = list(filter(None, test_ls))
    test_ls = np.array(test_ls).reshape(1, len(test_ls))
    x = padding_doc(test_ls, max_length)
    pred = model.predict_proba(x)
    return pred

#Function to output predictions
def get_final_output(pred, classes, mode):
    predictions = pred[0]
    classes = np.array(classes)
    ids = np.argsort(-predictions)
    classes = classes[ids]
    predictions = -np.sort(-predictions)
    if mode == 'rank':
        for i in range(pred.shape[1]):
            print("%s has confidence = %s" % (classes[i], (predictions[i])))
    elif mode == 'classify':
        return classes[0]

In [3]:
if __name__ == "__main__":
    #Extract Dataset
    filename = 'datasets/Preliminary_Dataset.csv'
    trainingdata = readdata(filename)
    messages = trainingdata['message']
    intent = trainingdata['Intent']
    unique_intent = list(set(intent))

In [33]:
    #Process x values
    #Cleaning messages, removing numbers and 
    words = cleaning(messages)

    #Tokenizing messages
    word_tokenizer = create_tokenizer(words)
    #Saving tokenizer for other files
    with open('tokenizer.pickle', 'wb') as handle:
        pickle.dump(word_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    #Figure out vocabulary size
    vocab_size = len(word_tokenizer.word_index) + 1

    #Find the max length of a message
    max_length = maximum_length(words)
    #print("Vocab Size = %d and Maximum length = %d" % (vocab_size, max_length))
    #Save the max length to a file
    with open('maxlen.txt', 'w') as file:
        file.write(str(max_length))
    
    #Created a document that can be inputted in Keras
    encoded_doc = encoding_doc(word_tokenizer,words)
    padded_doc = pad_sequences(encoded_doc, maxlen = max_length, padding = "post")
    #print(padded_doc)

In [19]:
    #Process Outputs
    #Tokenize intentions
    output_tokenizer=create_tokenizer(unique_intent,filters = '!"#$%&()*+,-/:;<=>?@[\]^`{|}~')
    #output_tokenizer.word_index

    #Number of unique intents
    intentsize = len(unique_intent)

    #Encode output for keras
    encoded_output =  encoding_doc(output_tokenizer,intent)
    encoded_output = np.array(encoded_output).reshape(len(encoded_output), 1)

    #make onehot output
    output_one_hot = one_hot(encoded_output)

    x_train, x_test, y_train, y_test = train_test_split(padded_doc, output_one_hot, shuffle = True, test_size=0.2, random_state=1000)
    #print("Shape of x_train = %s and y_train = %s" % (x_train.shape, y_train.shape))
    #print("Shape of x_test = %s and y_test = %s" % (x_test.shape, y_test.shape))

    print("Created test and training data.")

Created test and training data.


/home/laitcl/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [6]:
    #Model Creation
    model = create_model(vocab_size, max_length)

    model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 42, 128)           188544    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               263168    
_________________________________________________________________
dense_1 (Dense)              (None, 32)                8224      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 132       
Total params: 460,068
Trainable params: 271,524
Non-trainable params: 188,544
_________

In [7]:
    # Fit the model to the training data
    filename = 'model_weights.h5'
    checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

    hist = model.fit(x_train, y_train, epochs = 20, batch_size = 32, validation_data = (x_test, y_test), callbacks = [checkpoint])
    
    model.save_weights('model_weights.h5')
    # Save the model architecture
    with open('model_architecture.json', 'w') as f:
        f.write(model.to_json())    

Instructions for updating:
Use tf.cast instead.
Train on 2160 samples, validate on 540 samples
Epoch 1/20
2160/2160 [==============================] - 23s 11ms/step - loss: 1.2265 - acc: 0.4394 - val_loss: 1.0042 - val_acc: 0.5648

Epoch 00001: val_loss improved from inf to 1.00424, saving model to model_weights.h5
Epoch 2/20
2160/2160 [==============================] - 19s 9ms/step - loss: 0.9777 - acc: 0.5384 - val_loss: 0.8278 - val_acc: 0.5722

Epoch 00002: val_loss improved from 1.00424 to 0.82776, saving model to model_weights.h5
Epoch 3/20
2160/2160 [==============================] - 18s 8ms/step - loss: 0.8655 - acc: 0.6097 - val_loss: 0.7177 - val_acc: 0.6852

Epoch 00003: val_loss improved from 0.82776 to 0.71767, saving model to model_weights.h5
Epoch 4/20
2160/2160 [==============================] - 18s 8ms/step - loss: 0.7636 - acc: 0.6597 - val_loss: 0.6552 - val_acc: 0.7019

Epoch 00004: val_loss improved from 0.71767 to 0.65523, saving model to model_weights.h5
Epoch 5/

In [34]:
    # Load our proudly trained model
    with open('model_architecture.json', 'r') as f1:
        model = model_from_json(f1.read())

    # Load weights into the new model
    model.load_weights('model_weights.h5')

In [35]:
    #Example text
    text = "f"
    #Prediction
    pred = predictions(word_tokenizer, text, model, max_length)
    get_final_output(pred, unique_intent, 'classify')

'disappointment'

In [36]:
    #Export this notebook as a script
    #!jupyter nbconvert --to script Intent_Classification_Lai.ipynb

[NbConvertApp] Converting notebook Intent_Classification_Lai.ipynb to script
[NbConvertApp] Writing 6088 bytes to Intent_Classification_Lai.py
